In [2]:
#Import standard Python Libraries
import re
import pandas as pd
from datetime import datetime

#Parameters
input_fix_file = "trading.fix"
output_csv_file = "parsed_orders.csv"

def parse_fix_line(line: str):
    """Converts FIX line to a dictionary
    
    Args:
        line: str
        
    Returns:
        Dictionary mapping
        
    Raises:
        IndexError: If input line isn't able to be split
    """

    try:
        msg = line.split(":", 1)[1]
    except IndexError:
        return {}
    
    parts = msg.strip().split('\x01')
    fix_dict = {}
    for p in parts:
        if "=" in p:
            tag, val = p.split("=", 1)
            fix_dict[tag] = val
    return fix_dict

with open(input_fix_file, "r") as f:
    lines = f.readlines()
orders = {}
fills = []
for line in lines[:1]:
    fix = parse_fix_line(line)
fix

{'02:34.713753 : 8': 'FIX.4.2',
 '9': '330',
 '35': '8',
 '34': '1962',
 '49': 'ID1',
 '52': '20250910-04:02:34.713',
 '56': 'ID2',
 '1': 'ID6',
 '6': '32.98',
 '11': 'ID3',
 '14': '10',
 '17': 'ID5',
 '20': '0',
 '30': 'ID7',
 '31': '32.98',
 '32': '10',
 '37': 'ID4',
 '38': '10',
 '39': '2',
 '40': '2',
 '44': '32.98',
 '54': '2',
 '55': 'SMCX',
 '59': '0',
 '60': '20250910-04:02:34.713',
 '76': 'ID8',
 '150': '2',
 '151': '0',
 '10': '235'}

In [14]:
with open(input_fix_file, "r") as f:
    lines = f.readlines()
orders = {}
fills = []
for line in lines:
    fix = parse_fix_line(line)

    msg_type = fix.get("35")

    #Checks for New Order Single(D) and Limit Order (2)
    if msg_type == 'D' and fix['40'] == '2':
        client_order_id = fix['11']
        if client_order_id:
            orders[client_order_id] = {
                'OrderID': client_order_id,
                'OrderTransactTime': fix['60'],
                'Symbol': fix['55'],
                'Side': fix['54'],
                'OrderQty': fix['38'],
                'LimitPrice': fix['44'],
            }

    #Checks for MsgType (35) = Execution Report (8)
    elif msg_type == '8':
        # Check for Fully Filled limit orders, ignore partially filled limit orders
        if fix['150'] == '2' and fix['39'] == '2' and fix['40'] == '2':
            client_order_id = fix['11']
            if client_order_id in orders:
                order = orders[client_order_id]
                fills.append({
                    'OrderID': client_order_id,
                    'OrderTransactTime': order['OrderTransactTime'],
                    'ExecutionTransactTime': fix['60'],
                    'Symbol': order['Symbol'],
                    'Side': order['Side'],
                    'OrderQty': order['OrderQty'],
                    'LimitPrice': order['LimitPrice'],
                    'AvgPx': fix['6'],
                    'LastMkt': fix['30'],
                })

#Create dataframe of filled orders
filled_orders = pd.DataFrame(fills)
print("Parsed rows:", len(filled_orders))
display(filled_orders.head())

#Save to CSV file
filled_orders.to_csv(output_csv_file, index=False)
print(f"CSV saved to {output_csv_file}")

Parsed rows: 10


,OrderID,OrderTransactTime,ExecutionTransactTime,Symbol,Side,OrderQty,LimitPrice,AvgPx,LastMkt
0,ID1542,20250910-08:00:00.377,20250910-08:00:00.509,YANG,1,27,23.7,23.170000,ID1516
1,ID1572,20250910-08:00:00.450,20250910-08:00:00.692,IREN,1,30,31.29,31.290000,ID1516
2,ID1632,20250910-08:00:00.747,20250910-08:00:00.982,DJT,2,53,16.91,16.910000,ID1516
3,ID1711,20250910-08:00:00.866,20250910-08:00:01.006,AMD,2,80,159.56,160.110000,ID1516
4,ID1732,20250910-08:00:00.920,20250910-08:00:01.042,SOFI,2,4,25.86,25.890000,ID1516


CSV saved to parsed_orders.csv


In [ ]:
msg_type = fix['35']
#Checks for New Order Single (D)
f msg_type == "D" and fix.get("40") == "2":  # Limit orders only
        clid = fix.get("11")
        if clid:
            orders[clid] = {
                "OrderID": clid,
                "OrderTransactTime": fix.get("60"),
                "Symbol": fix.get("55"),
                "Side": fix.get("54"),
                "OrderQty": fix.get("38"),
                "LimitPrice": fix.get("44"),
            }

'8'

In [6]:
fix.get('11')

'ID2052'